In [1]:
import torch
import os
import pandas as pd
import numpy as np
from ocpmodels.trainers import EnergyTrainer, ForcesTrainer
from ocpmodels.datasets import SinglePointLmdbDataset
from ocpmodels import models
from ocpmodels.common import logger
from ocpmodels.common.utils import setup_logging
setup_logging()

D:\SA\project\fairchem\ocpmodels\models\equiformer_v2\wigner.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd = torch.load(os.path.join(os.path.dirname(__file__), "Jd

In [2]:
task = {
    'dataset': 'single_point_lmdb', 
    'description': 'Regressing to energies and forces for DFT trajectories from OCP',
    'type': 'regression',
    'metric': 'mae',
    'labels': ['potential energy'],
    'grad_input': 'atomic forces',
    'train_on_free_atoms': True,
    'eval_on_free_atoms': True
}
model = {  
  'name': 'cgcnn',
  'atom_embedding_size': 384,
  'fc_feat_size': 512,
  'num_fc_layers': 4,
  'num_graph_conv_layers': 6,
  'num_gaussians': 100,
  'cutoff': 6.0,
  'regress_forces': False,
  'use_pbc': True,
}
optimizer = {
    'batch_size': 32,
  'eval_batch_size': 32,
  'num_workers': 0,
  'lr_initial': 0.0005,
    'scheduler': "ReduceLROnPlateau",
    'mode': "min",
    'factor': 0.5,
    'patience': 3,
    'max_epochs': 60,
}
metal_dict = {}

In [3]:
facet_list = ['100','110','111']
alloy = 'zn'

for facet in facet_list:
    for n in range(1,4):
        metal_dict['cu'+facet+'_'+alloy+str(n/10)] =  {}
        for ad in ['co','oh']:
            test_src = '/Users/paynehu/Desktop/FYP/notebook/cu-'+alloy+'/cu-'+alloy+'/cu'+facet+ad+'_'+alloy+str(n/10)+'.lmdb'
            dataset = [{'src': test_src, 'normalize_labels': False},
                {'src': test_src}, {'src': test_src}]
            task = {
                'dataset': 'single_point_lmdb', 
                'description': 'Regressing to energies and forces for DFT trajectories from OCP',
                'type': 'regression',
                'metric': 'mae',
                'labels': ['potential energy'],
                'grad_input': 'atomic forces',
                'train_on_free_atoms': True,
                'eval_on_free_atoms': True
            }
            model = {  
            'name': 'cgcnn',
            'atom_embedding_size': 384,
            'fc_feat_size': 512,
            'num_fc_layers': 4,
            'num_graph_conv_layers': 6,
            'num_gaussians': 100,
            'cutoff': 6.0,
            'regress_forces': False,
            'use_pbc': True,
            }
            optimizer = {
                'batch_size': 32,
            'eval_batch_size': 32,
            'num_workers': 0,
            'lr_initial': 0.0005,
                'scheduler': "ReduceLROnPlateau",
                'mode': "min",
                'factor': 0.5,
                'patience': 3,
                'max_epochs': 60,
            }
            pretrained_trainer = EnergyTrainer(
                task=task,
                model=model,
                dataset=dataset,
                optimizer=optimizer,
                identifier="cgcnn_predict",
                run_dir="./", # directory to save results if is_debug=False. Prediction files are saved here so be careful not to override!
                is_debug=False, # if True, do not save checkpoint, logs, or results
                #is_vis=False,
                print_every=10,
                seed=0, # random seed to use
                logger="tensorboard", # logger of choice (tensorboard and wandb supported)
                local_rank=0,
                amp=False, # use PyTorch Automatic Mixed Precision (faster training and less memory usage)
                )
            if ad == 'co':
                state_dict = torch.load('checkpoint_co.pt',map_location=torch.device('cpu'))
                print(ad, facet)
            #elif cat == 'h':
                #state_dict = torch.load('checkpoint_h.pt',map_location=torch.device('cpu'))
                #print(cat, facet)
            elif ad == 'oh':
                state_dict = torch.load('checkpoint_oh.pt',map_location=torch.device('cpu'))
                print(ad, facet)
            pretrained_trainer.model.load_state_dict(state_dict['state_dict'])
            predictions = pretrained_trainer.predict(pretrained_trainer.test_loader, results_file= 'bimetallic_results', disable_tqdm=False)
            energies = predictions["energy"]
            metal_dict['cu'+facet+'_'+alloy+str(n/10)][ad] = energies

2024-10-02 15:18:31 (INFO): amp: false
cmd:
  checkpoint_dir: ./checkpoints\2024-10-02-15-19-28-cgcnn_predict
  commit: 6f24a48b
  identifier: cgcnn_predict
  logs_dir: ./logs\tensorboard\2024-10-02-15-19-28-cgcnn_predict
  print_every: 10
  results_dir: ./results\2024-10-02-15-19-28-cgcnn_predict
  seed: 0
  timestamp_id: 2024-10-02-15-19-28-cgcnn_predict
dataset:
  normalize_labels: false
  src: /Users/paynehu/Desktop/FYP/notebook/cu-zn/cu-zn/cu100co_zn0.1.lmdb
gpus: 0
logger: tensorboard
model: cgcnn
model_attributes:
  atom_embedding_size: 384
  cutoff: 6.0
  fc_feat_size: 512
  num_fc_layers: 4
  num_gaussians: 100
  num_graph_conv_layers: 6
  regress_forces: false
  use_pbc: true
noddp: false
optim:
  batch_size: 32
  eval_batch_size: 32
  factor: 0.5
  lr_initial: 0.0005
  max_epochs: 60
  mode: min
  num_workers: 0
  patience: 3
  scheduler: ReduceLROnPlateau
slurm: {}
task:
  dataset: single_point_lmdb
  description: Regressing to energies and forces for DFT trajectories from 

AssertionError: No LMDBs found in '\Users\paynehu\Desktop\FYP\notebook\cu-zn\cu-zn\cu100co_zn0.1.lmdb'

In [24]:
original_test_result = pd.DataFrame.from_dict(metal_dict).T
original_test_result.to_csv('/Users/paynehu/Desktop/FYP/notebook/original_test_result_bi.csv')
original_test_result

,co,oh
cu100_ni0.1,"[-0.572304904460907, -1.2007237672805786, -0.7...","[-0.13485030829906464, 0.02636316604912281, -0..."
cu100_ni0.2,"[-0.5957820415496826, -0.5353075861930847, -0....","[0.1805981695652008, 0.023505961522459984, -0...."
cu100_ni0.3,"[-0.5880486965179443, -0.5662444233894348, -0....","[0.4047892093658447, -0.10422606021165848, -0...."
cu100_ni0.4,"[-0.5951992869377136, -1.3103326559066772, -0....","[0.33334580063819885, -0.08111388236284256, -0..."
cu100_ni0.5,"[-1.5450383424758911, -1.5262917280197144, -1....","[-0.035540979355573654, 0.14998698234558105, -..."
cu110_ni0.1,"[-0.6918084025382996, -0.41711801290512085, -0...","[-0.04752478003501892, -0.12606394290924072, -..."
cu110_ni0.2,"[-0.7621058821678162, -0.7929303050041199, -0....","[-0.08559809625148773, -0.1519967019557953, -0..."
cu110_ni0.3,"[-0.6933537125587463, -0.028834259137511253, -...","[-0.018350981175899506, -0.4632718861103058, -..."
cu110_ni0.4,"[-0.7784247994422913, -0.013360694982111454, -...","[-0.021952925249934196, -0.4178527295589447, -..."
cu110_ni0.5,"[-1.6279443502426147, -1.0707457065582275, -1....","[-0.22587502002716064, -0.09356991201639175, -..."


In [28]:
std_test_result = original_test_result.applymap(np.std)
std_test_result

,co,oh
cu100_ni0.1,0.321667,0.209402
cu100_ni0.2,0.361457,0.232571
cu100_ni0.3,0.407527,0.260554
cu100_ni0.4,0.385096,0.272601
cu100_ni0.5,0.395784,0.277013
cu110_ni0.1,0.372280,0.215187
cu110_ni0.2,0.434286,0.246628
cu110_ni0.3,0.531984,0.263026
cu110_ni0.4,0.596403,0.286945
cu110_ni0.5,0.614872,0.284220


In [27]:
average_test_result = original_test_result.applymap(np.mean)
average_test_result

,co,oh
cu100_ni0.1,-0.560009,0.075866
cu100_ni0.2,-0.732441,0.070705
cu100_ni0.3,-0.897687,0.060678
cu100_ni0.4,-1.082348,0.067248
cu100_ni0.5,-1.185794,0.012452
cu110_ni0.1,-0.589508,-0.289344
cu110_ni0.2,-0.624811,-0.277210
cu110_ni0.3,-0.749419,-0.298701
cu110_ni0.4,-0.827965,-0.275972
cu110_ni0.5,-0.899213,-0.330115


In [29]:
average_test_result['co'] = average_test_result['co'].apply(lambda x: round(x+0.5,3))
#average_test_result['h'] = average_test_result['h'].apply(lambda x: round(x+0.24,3))
average_test_result['oh'] = average_test_result['oh'].apply(lambda x: round(x+0.35,3))

In [30]:
average_test_result.to_csv('/Users/paynehu/Desktop/FYP/notebook/average_test_result_bi.csv')

In [ ]:
eU = -1.18
fig, ax = plt.subplots(1,1)
gco = np.arange(-10, 10)
goh = np.arange(-1.5,1.5)
plt.rcParams["font.weight"] = "bold"
plt.rcParams["font.size"] = 8
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams["axes.labelsize"] = 8

plt.xlabel(r'$G_{CO^*}(eV)$', fontweight = 'bold', fontsize = 12)
plt.ylabel(r'$G_{OH^*}(eV)$', fontweight = 'bold', fontsize = 12)

plt.ylim(top=1.5) 
plt.ylim(bottom=-1.5)
plt.xlim(left=-1.5) 
plt.xlim(right=1.0)

goh = (1.47*gco+1.15+eU)/0.81 #eq1
plt.plot(gco,(1.47*gco+1.15+eU)/0.81,label='eq.1',color='g',linestyle = 'dashed', linewidth=2)

eq2 = plt.axvline(x=(-eU-0.27)/0.58, label='eq.2',color='g',linestyle = 'dashed', linewidth=2)
eq3 = plt.axhline(y=eU,label='eq.3',color='b',linestyle = 'dashed', linewidth=2)

gco=(-eU-0.66)/2.22
ymin=((1.47*gco+1.15+eU)/0.81+1.5)/3

eq4 = plt.axvline(gco, ymin=ymin, label='eq.4',color='r',linestyle = 'dashed', linewidth=2)#check

gco=0.35/-1.09
ymin=((1.47*gco+1.15+eU)/0.81+1.5)/3

eq5 = plt.axvline(gco, ymin=ymin, label='eq.5',color='k',linestyle = 'dashed', linewidth=2)

gco=(0.27-eU)/-0.58

eq6 = plt.axvline(gco, label='eq.6',color='b',linestyle = 'dashed', linewidth=2)


x1 = [(-eU-0.27)/0.58,(-eU-0.27)/0.58,(0.81*eU-1.15-eU)/1.47]
y1 = [eU,(1.47*((-eU-0.27)/0.58)+1.15+eU)/0.81,eU]
ax.fill(x1,y1, color='g',alpha=.1)

x2 = [-1.5,(0.81*eU-1.15-eU)/1.47,0.35/-1.09,0.35/-1.09,-1.5]
y2 = [eU,eU,(1.47*(0.35/-1.09)+1.15+eU)/0.81,1.5,1.5]
ax.fill(x2,y2, color='b',alpha=.1)

x3 = [0.35/-1.09,0.35/-1.09,(-eU-0.66)/2.22,(-eU-0.66)/2.22]
y3 = [(1.47*(0.35/-1.09)+1.15+eU)/0.81,1.5,1.5,(1.47*((-eU-0.66)/2.22)+1.15+eU)/0.81]
ax.fill(x3,y3, color='r',alpha=.1)

x4 = [(-eU-0.66)/2.22,(-eU-0.66)/2.22,(-eU-0.27)/0.58,(-eU-0.27)/0.58]
y4 = [1.5,(1.47*((-eU-0.66)/2.22)+1.15+eU)/0.81,(1.47*((-eU-0.27)/0.58)+1.15+eU)/0.81,1.5]
ax.fill(x4,y4, color='y',alpha=.1)

a = 0
Cu100 = ax.plot(xy2.iloc[3,0],xy2.iloc[3,1], color = '#ffff00', marker='o')
Cu100_Ni1 = ax.plot(xy.iloc[a,0],xy.iloc[a,1], color = '#e1e300', marker='o')
Cu100_Ni2 = ax.plot(xy.iloc[a+1,0],xy.iloc[a+1,1], color = '#c3c800', marker='o')
Cu100_Ni3 = ax.plot(xy.iloc[a+2,0],xy.iloc[a+2,1], color = '#a6ae00', marker='o')
Cu100_Ni4 = ax.plot(xy.iloc[a+3,0],xy.iloc[a+3,1], color = '#899400', marker='o')
Cu100_Ni5 = ax.plot(xy.iloc[a+4,0],xy.iloc[a+4,1], color = '#6e7b00', marker='o')
Ni100 = ax.plot(xy2.iloc[6,0],xy2.iloc[6,1], color = 'k', marker='o')

a = 5
Cu110 = ax.plot(xy2.iloc[4,0],xy2.iloc[4,1], color = '#ffff00', marker='s')
Cu110_Ni1 = ax.plot(xy.iloc[a,0],xy.iloc[a,1], color = '#e1e300', marker='s')
Cu110_Ni2 = ax.plot(xy.iloc[a+1,0],xy.iloc[a+1,1], color = '#c3c800', marker='s')
Cu110_Ni3 = ax.plot(xy.iloc[a+2,0],xy.iloc[a+2,1], color = '#a6ae00', marker='s')
Cu110_Ni4 = ax.plot(xy.iloc[a+3,0],xy.iloc[a+3,1], color = '#899400', marker='s')
Cu110_Ni5 = ax.plot(xy.iloc[a+4,0],xy.iloc[a+4,1], color = '#6e7b00', marker='s')
Ni110 = ax.plot(xy2.iloc[7,0],xy2.iloc[7,1], color = 'k', marker='s')

a = 10
Cu111 = ax.plot(xy2.iloc[5,0],xy2.iloc[5,1], color = '#ffff00', marker='^')
Cu111_Ni1 = ax.plot(xy.iloc[a,0],xy.iloc[a,1], color = '#e1e300', marker='^')
Cu111_Ni2 = ax.plot(xy.iloc[a+1,0],xy.iloc[a+1,1], color = '#c3c800', marker='^')
Cu111_Ni3 = ax.plot(xy.iloc[a+2,0],xy.iloc[a+2,1], color = '#a6ae00', marker='^')
Cu111_Ni4 = ax.plot(xy.iloc[a+3,0],xy.iloc[a+3,1], color = '#899400', marker='^')
Cu111_Ni5 = ax.plot(xy.iloc[a+4,0],xy.iloc[a+4,1], color = '#6e7b00', marker='^')
Ni111 = ax.plot(xy2.iloc[8,0],xy2.iloc[8,1], color = 'k', marker='^')

a = 15
Cu100 = ax.plot(xy2.iloc[3,0],xy2.iloc[3,1], color = '#ffff00', marker='o')
Cu100_Zn1 = ax.plot(xy.iloc[a,0],xy.iloc[a,1], color = '#cccc33', marker='o')
Cu100_Zn2 = ax.plot(xy.iloc[a+1,0],xy.iloc[a+1,1], color = '#999966', marker='o')
Cu100_Zn3 = ax.plot(xy.iloc[a+2,0],xy.iloc[a+2,1], color = '#666699', marker='o')
Zn100 = ax.plot(xy2.iloc[14,0],xy2.iloc[14,1], color = 'b', marker='o')

a = 18
Cu110 = ax.plot(xy2.iloc[4,0],xy2.iloc[4,1], color = '#ffff00', marker='s')
Cu110_Zn1 = ax.plot(xy.iloc[a,0],xy.iloc[a,1], color = '#cccc33', marker='s')
Cu110_Zn2 = ax.plot(xy.iloc[a+1,0],xy.iloc[a+1,1], color = '#999966', marker='s')
Cu110_Zn3 = ax.plot(xy.iloc[a+2,0],xy.iloc[a+2,1], color = '#666699', marker='s')


a = 21
Cu111 = ax.plot(xy2.iloc[5,0],xy2.iloc[5,1], color = '#ffff00', marker='^')
Cu111_Zn1 = ax.plot(xy.iloc[a,0],xy.iloc[a,1], color = '#cccc33', marker='^')
Cu111_Zn2 = ax.plot(xy.iloc[a+1,0],xy.iloc[a+1,1], color = '#999966', marker='^')
Cu111_Zn3 = ax.plot(xy.iloc[a+2,0],xy.iloc[a+2,1], color = '#666699', marker='^')


ax.xaxis.set_major_locator(MultipleLocator(0.5))
ax.xaxis.set_minor_locator(MultipleLocator(0.1))

ax.yaxis.set_major_locator(MultipleLocator(0.5))
ax.yaxis.set_minor_locator(MultipleLocator(0.1))

black =  Patch(color = 'k', label='Ni')
yellow =  Patch(color = 'y', label='Cu')
blue =  Patch(color = 'b', label='Zn')

circle = mlines.Line2D([], [], marker='o', linestyle='None',label = '(100)')
square = mlines.Line2D([], [], marker='s', linestyle='None',label = '(110)')
triangle_up = mlines.Line2D([], [], marker='^', linestyle='None',label = '(111)')

handles =[circle,square,triangle_up]

first_legend = plt.legend(handles = handles,edgecolor = 'k',loc='center left', bbox_to_anchor=(1.05,0.2), ncol=1)
second_legend = plt.legend(handles = [yellow,black,blue],edgecolor = 'k',loc='center left', bbox_to_anchor=(1.05,0.5), ncol=1)
plt.gca().add_artist(first_legend)
plt.gca().add_artist(second_legend)

red =  Patch(color = 'r',alpha = .3, label=r'$C_{1+}(>2e^-)$')
yellow =  Patch(color = 'y', alpha = .3, label='CO')
blue =  Patch(color = 'b', alpha = .3, label='H')
green =  Patch(color = 'g', alpha = .3, label='Formate')
third_legend = plt.legend(handles = [blue,red,yellow,green], edgecolor = 'k',loc='center left', bbox_to_anchor=(1.05,0.8), ncol=1)
plt.text(-1.4,1.2,'U = -1.18 V',fontsize = 12).set_bbox(dict(facecolor='white'))
plt.show()
fig.savefig('-1.18 V bi.png', format='png', dpi=800, bbox_inches='tight')